In [1]:
import pandas as pd

### Get gene to protein sequence mapping

In [2]:
with open("../../data/raw/9606.protein.sequences.v11.5.fa") as f:
    lines = f.readlines()
    line_str = "".join(lines)
id_seq_str_list = line_str.split(">")
prot_id_to_seq = {}
for id_seq_str in id_seq_str_list:
    tokens = id_seq_str.split("\n")
    key = tokens[0]
    val = "".join(tokens[1:])
    prot_id_to_seq[key] = val

In [3]:
prot_info_df = pd.read_csv("../../data/raw/9606.protein.info.v11.5.txt", "\t")
prot_info_gene = prot_info_df["preferred_name"].unique()
prot_info_df["preferred_name"].nunique()

/root/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


19563

In [4]:
prot_info_df["proteinSeq"] = prot_info_df["#string_protein_id"].apply(
    lambda x: prot_id_to_seq[x]
)

In [5]:
gene_dict = {}
for _, item in prot_info_df.iterrows():
    gene_dict[item["preferred_name"]] = item["proteinSeq"]

### Get disease name to disease des mapping

In [6]:
full_gda = pd.read_csv("../../data/pretrain/disgenet_gda.csv")
full_gda.head()
disease_dict = {}
for _, item in full_gda.iterrows():
    disease_dict[item["diseaseName"]] = item["diseaseDes"]

### mapp training data to sequences

In [8]:
train_df = pd.read_csv("../../data/downstream/gnn_feature_v1_samples_train.csv")
train_df["proteinSeq"] = train_df["gnn_feature_v1_samples_train.protein_name"].apply(
    lambda x: gene_dict[x]
)
train_df["diseaseDes"] = train_df["gnn_feature_v1_samples_train.diseasename"].apply(
    lambda x: disease_dict[x] if x in disease_dict else x
)
train_df = train_df.rename(
    columns={
        "gnn_feature_v1_samples_train.protein_name": "protein_name",
        "gnn_feature_v1_samples_train.diseasename": "diseasename",
        "gnn_feature_v1_samples_train.protein_feature":"protein_feature",
        "gnn_feature_v1_samples_train.disease_feature":"disease_feature",
        "gnn_feature_v1_samples_train.label":"label"
    }
)
train_df = train_df[["protein_name", "proteinSeq","protein_feature", "diseasename", "diseaseDes","disease_feature","label"]]
train_df.to_csv("../../data/downstream/disgenet_gda_cls_train.csv", index=None)

In [9]:
val_df = pd.read_csv("../../data/downstream/gnn_feature_v1_samples_val.csv")
val_df["proteinSeq"] = val_df["gnn_feature_v1_samples_val.protein_name"].apply(
    lambda x: gene_dict[x]
)
val_df["diseaseDes"] = val_df["gnn_feature_v1_samples_val.diseasename"].apply(
    lambda x: disease_dict[x] if x in disease_dict else x
)
val_df = val_df.rename(
    columns={
        "gnn_feature_v1_samples_val.protein_name": "protein_name",
        "gnn_feature_v1_samples_val.diseasename": "diseasename",
        "gnn_feature_v1_samples_val.protein_feature":"protein_feature",
        "gnn_feature_v1_samples_val.disease_feature":"disease_feature",
         "gnn_feature_v1_samples_val.label":"label"
    }
)
val_df = val_df[["protein_name", "proteinSeq","protein_feature", "diseasename", "diseaseDes","disease_feature","label"]]
val_df.to_csv("../../data/downstream/disgenet_gda_cls_val.csv", index=None)

In [10]:
test_df = pd.read_csv("../../data/downstream/gnn_feature_v1_samples_test.csv")
test_df["proteinSeq"] = test_df["gnn_feature_v1_samples_test.protein_name"].apply(
    lambda x: gene_dict[x]
)
test_df["diseaseDes"] = test_df["gnn_feature_v1_samples_test.diseasename"].apply(
    lambda x: disease_dict[x] if x in disease_dict else x
)
test_df = test_df.rename(
    columns={
        "gnn_feature_v1_samples_test.protein_name": "protein_name",
        "gnn_feature_v1_samples_test.diseasename": "diseasename",
        "gnn_feature_v1_samples_test.protein_feature":"protein_feature",
        "gnn_feature_v1_samples_test.disease_feature":"disease_feature",
        "gnn_feature_v1_samples_test.label":"label"
    }
)
test_df = test_df[["protein_name", "proteinSeq","protein_feature", "diseasename", "diseaseDes","disease_feature","label"]]
test_df.to_csv("../../data/downstream/disgenet_gda_cls_test.csv", index=None)

In [11]:
test_df

,protein_name,proteinSeq,protein_feature,diseasename,diseaseDes,disease_feature,label
0,XPR1,MKFAEHLSAHITPEWRKQYIQYEAFKDMLYSAQDQAPSVEVTDEDT...,"{""0"": 1, ""43901"": 0.010086, ""20"": 1, ""817"": 1,...","Coma, Post-Head Injury",Prolonged unconsciousness from which the indiv...,"{""43968"": 1, ""1"": 1, ""43975"": 1, ""44872"": 1, ""...",0
1,PTDSS1,MASCVGSRTLSKDDVNYKMHFRMINEQQVEDITIDFFYRPHTITLL...,"{""0"": 1, ""43905"": 0.006369, ""19"": 1, ""43872"": ...","Purine-Pyrimidine Metabolism, Inborn Errors",Dysfunctions in the metabolism of PURINES or P...,"{""1"": 1, ""43986"": 1, ""44838"": 1, ""44689"": 1, ""...",0
2,RP1L1,MNSTPRNAQAPSHRECFLPSVARTPSVTKVTPAKKITFLKRGDPRF...,"{""0"": 1, ""2"": 1, ""543"": 1, ""299"": 1, ""43911"": ...",Spinal muscular atrophy 4,Spinal muscular atrophy 4,"{""44832"": 1, ""1"": 1, ""44834"": 1, ""44913"": 1, ""...",0
3,AZGP1,MVRMVPVLLSLLLLLGPAVPQENQDGRYSLTYIYTGLSKHVEDVPA...,"{""0"": 1, ""769"": 1, ""518"": 1, ""520"": 1, ""528"": ...",Myofibromatosis,A mesenchymal neoplasm characterized by solita...,"{""1"": 1, ""44788"": 1, ""44774"": 1, ""44777"": 1, ""...",0
4,AHCYL2,MSVQVVSAAAAAKVPEVELKDLSPSEAESQLGLSTAAVGAMAPPAG...,"{""0"": 1, ""43906"": 0.024631, ""278"": 1, ""535"": 1...",Lymphangioleiomyomatosis,Lymphangioleiomyomatosis (LAM) is a condition ...,"{""1"": 1, ""44034"": 1, ""44774"": 1, ""44678"": 1, ""...",0
...,...,...,...,...,...,...,...
4387,STAT1,MSQWYELQQLDSKFLEQVHQLYDDSFPMEIRQYLAQWLEKQDWEHA...,"{""0"": 1, ""1025"": 1, ""773"": 1, ""6"": 1, ""43901"":...",Urinary Retention,Inability to completely empty the urinary blad...,"{""1"": 1, ""44236"": 1, ""44554"": 1, ""44140"": 1, ""...",0
4388,NEK4,MPLAAYCYLRVVGKGSYGEVTLVKHRRDGKQYVIKKLNLRNASSRE...,"{""0"": 1, ""1025"": 1, ""43927"": 0.009535, ""28"": 1...",Dicarboxylicaminoaciduria,Characterized by infantile-onset hypoglycemia ...,"{""1"": 1, ""44162"": 1, ""44679"": 1, ""44297"": 1, ""...",0
4389,LRPPRC,MAALLRSARWLLRAGAAPRLPLSLRLLPGGPGRLHAASYLPAARAG...,"{""0"": 1, ""3"": 1, ""43905"": 0.006466, ""43919"": 0...",Lethal Congenital Contracture Syndrome 2,Lethal congenital contracture syndrome type 2 ...,"{""1"": 1, ""44150"": 1, ""44151"": 1, ""44107"": 1, ""...",0
4390,CYTH1,MEEDDSYVPSDLTAEERQELENIRRRKQELLADIQRLKDEIAEVAN...,"{""0"": 1, ""523"": 1, ""279"": 1, ""306"": 1, ""63"": 1...",Lesch-Nyhan Syndrome,"HPRT1 disorders, caused by deficiency of the e...","{""1"": 1, ""44931"": 1, ""44556"": 1, ""44689"": 1, ""...",0


In [12]:
import pandas as pd
test_df = pd.read_csv("../../data/../data/downstream/disgenet_gda_cls_test.csv")

In [13]:
test_df.label.value_counts()

0    4023
1     369
Name: label, dtype: int64

In [14]:
test_df.sample(frac=1).reset_index(drop=True)

,protein_name,proteinSeq,protein_feature,diseasename,diseaseDes,disease_feature,label
0,ZC3H12A,MSGPCGEKPVLEASPTMSLWEFEDSHSRQGTPRPGQELAAEEASAL...,"{""0"": 1, ""769"": 1, ""3"": 1, ""43905"": 0.0067, ""4...",Leukotriene C4 Synthase Deficiency,Leukotriene C4 Synthase Deficiency,"{""1"": 1, ""43986"": 1, ""44838"": 1, ""44689"": 1, ""...",0
1,GDF1,MPPPQQGPCGHHLLLLLALLLPSLPLTRAPVPPGPAAALLQALGLR...,"{""0"": 1, ""43904"": 0.013514, ""43906"": 0.021622,...",Adams Oliver syndrome,Adams-Oliver syndrome (AOS) is characterized b...,"{""1"": 1, ""44233"": 1, ""44234"": 1, ""44236"": 1, ""...",0
2,SERPINC1,MYSNVIGTVTSGKRKVYLLSLLLIGFWDCVTCHGSPVDICTAKPRD...,"{""0"": 1, ""769"": 1, ""264"": 1, ""785"": 1, ""278"": ...",Uterine Neoplasms,A tumor (abnormal growth of tissue) of the ute...,"{""1"": 1, ""44481"": 1, ""44305"": 1, ""44370"": 1, ""...",0
3,MYO7A,MVILQQGDHVWMDLRLGQEFDVPIGAVVKLCDSGQVQVVDDEDNEH...,"{""0"": 1, ""43894"": 0.026209, ""263"": 1, ""270"": 1...",Cardiovascular Diseases,A non-neoplastic or neoplastic disorder affect...,"{""1"": 1, ""44479"": 1}",0
4,ZNF526,MAEVVAEVAEMPTQMSPGAVEMSTPMSAEMMEMSTEVTEMTPGEAL...,"{""0"": 1, ""43885"": 0.008982, ""43919"": 0.020958,...",Retinal Dystrophies,Retinal dystrophy is an abnormality of the ret...,"{""1"": 1, ""44442"": 1, ""43980"": 1, ""44057"": 1}",0
...,...,...,...,...,...,...,...
4387,IRAK4,MNKPITPSTYVRCLNVGLIRKLSDFIDPQEGWKKLAVAIKKPSGDD...,"{""0"": 1, ""43904"": 0.008734, ""2"": 1, ""522"": 1, ...",Keratosis,Excessive growth of keratin on the skin.,"{""1"": 1, ""44180"": 1, ""44158"": 1}",0
4388,CCT3,MMGHRPVLVLSQNTKRESGRKVQSGNINAAKTIADIIRTCLGPKSM...,"{""0"": 1, ""31"": 1, ""48"": 1, ""306"": 1, ""309"": 1,...",Pleural Diseases,A non-neoplastic or neoplastic disorder that a...,"{""1"": 1, ""44788"": 1}",0
4389,TPM1,MDAIKKKMQMLKLDKENALDRAEQAEADKKAAEDRSKQLEDELVSL...,"{""0"": 1, ""265"": 1, ""522"": 1, ""781"": 1, ""784"": ...","Atrial Fibrillation, Familial, 3","Atrial Fibrillation, Familial, 3","{""44800"": 1, ""1"": 1, ""44770"": 1, ""44134"": 1, ""...",0
4390,COG7,MDFSKFLADDFDVKEWINAAFRAGSKEAASGKADGHAATLVMKLQL...,"{""0"": 1, ""43884"": 0.005208, ""43910"": 0.005208,...",Congenital disorder of glycosylation type 2E,Congenital disorder of glycosylation type 2E,"{""1"": 1, ""44838"": 1, ""44427"": 1, ""43986"": 1, ""...",1
